This notebook describes the temporal fusion transformers [@lim2021temporal] architecture, and ports it over to keras 3 while making some punctual improvements, including making the notation closer to the paper math.

The original repository is: https://github.com/google-research/google-research/tree/master/tft.

In [2]:
from __future__ import annotations

import numpy as np
import keras_core as keras
from keras_core import layers
from fastcore import docments
from nbdev.showdoc import show_doc


Using TensorFlow backend


# Introduction

The main characteristics of TFT that make it interesting for nowcasting or forecasting purposes are:

- **multi-horizon forecasting**: the ability to output, at each point in time $t$, a sequence of forecasts for $t+h, h > 1$
- **quantile prediction**: each forecast is accompanied by a quantile band that communicates the amount of uncertainty around a prediction
- **flexible use of different types of inputs**: static inputs (akin to fixed effects), historical input and known future input (eg, important holidays, years that are known to have major sports events such as Olympic games, etc)
- **interpretability**: the model learns to select variables from the space of all input variables to retain only those that are globally meaningful, to assign attention to different parts of the time series, and to identify events of significance


# Architecture

## Concepts

* **time distributed**: 
  * applies same layer to each of the timesteps in the data
    * in other words, a layer with the exact same weights
  * indices:
    * index 0: batch
    * index 1: time
    * indices 2...: data
  * More info: https://www.tensorflow.org/api_docs/python/tf/keras/layers/TimeDistributed

## Components

### Linear layer

> dedicated implementation to better control use of time distribution on vanilla linear layer

$$
\mathbb{Y} = \phi(\mathbf{W} x + \mathbf{b}),
$$ {#eq-dense}

where $x$ is the input to `linear_layer()(x)`, $\mathbb{Y}$ is the output of `linear_layer()(x)`, $\phi$ is an activation function (or no activation function is `activation` is `None`), $\mathbf{W} \in \mathbf{R}^{(d_{\text{size}} \times d_{\text{inputs}})}$ is a matrix of weights and $\mathbf{b} \in \mathbf{R}^{d_{size}}$ is a vector of biases. Importantly, $\mathbf{W}$ and $\mathbf{b}$ are indexed with $_{\omega}$ to denote weight-sharing when the layer is time-distributed.

In [22]:
def linear_layer(size:int, # Output size
                 activation:str|callable|None=None, # Activation function
                 use_time_distributed:bool=False, # Apply the layer across all time steps?
                 use_bias:bool=True # Include bias in the layer?
)->layers.Dense: # Dense layer
    "Linear layer."

    linear = layers.Dense(size, activation=activation, use_bias=use_bias)
    if use_time_distributed:
        linear = layers.TimeDistributed(linear)
    return linear

In [51]:
#| output: asis
#| echo: false
show_doc(linear_layer, title_level=4)

---

#### linear_layer

>      linear_layer (size:int, activation:Union[str,<built-
>                    infunctioncallable>,NoneType]=None,
>                    use_time_distributed:bool=False, use_bias:bool=True)

Linear layer.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| size | int |  | Output size |
| activation | str \| callable \| None | None | Activation function |
| use_time_distributed | bool | False | Apply the layer across all time steps? |
| use_bias | bool | True | Include bias in the layer? |
| **Returns** | **layers.Dense** |  | **Dense layer** |

#### Example usage

In [24]:
#| code-fold: show

batch_size = 2
n_timesteps = 5
n_features = 100
layer_size = 8

# input dimensions: batches / timesteps / features
x = np.ones(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

# dense layer
linear_td_true = linear_layer(size=layer_size, use_time_distributed=True)
linear_td_false = linear_layer(size=layer_size, use_time_distributed=False)

# output dimensions: batches / timesteps / layer size
assert linear_td_true(x).shape == [batch_size, n_timesteps, layer_size]
assert linear_td_false(x).shape == [batch_size, n_timesteps, layer_size]

Note that the time-distributed linear layer results in the same weights being applied to each time step:

In [26]:
#| code-fold: show

x = np.ones((1, n_timesteps, n_features))
timesteps_equal = []
for i in range(n_timesteps-1):
    timesteps_equal.append((np.array_equal(linear_td_true(x)[0,0,:], linear_td_true(x)[0,i+1,:])))

assert np.all(timesteps_equal)

linear_td_true(x)

<tf.Tensor: shape=(1, 5, 8), dtype=float32, numpy=
array([[[-2.1470957 ,  1.0115887 , -0.6940057 ,  0.9228788 ,
          0.31520468, -0.12253807, -0.6000036 ,  1.6961169 ],
        [-2.1470957 ,  1.0115887 , -0.6940057 ,  0.9228788 ,
          0.31520468, -0.12253807, -0.6000036 ,  1.6961169 ],
        [-2.1470957 ,  1.0115887 , -0.6940057 ,  0.9228788 ,
          0.31520468, -0.12253807, -0.6000036 ,  1.6961169 ],
        [-2.1470957 ,  1.0115887 , -0.6940057 ,  0.9228788 ,
          0.31520468, -0.12253807, -0.6000036 ,  1.6961169 ],
        [-2.1470957 ,  1.0115887 , -0.6940057 ,  0.9228788 ,
          0.31520468, -0.12253807, -0.6000036 ,  1.6961169 ]]],
      dtype=float32)>

### Skip connection

> Adds inputs to layer and then implements layer normalisation

$$
\text{LayerNorm}(a + b),
$$ {#eq-skip}

for $a$ and $b$ tensors of the same dimension and $\text{LayerNorm}(\cdot)$ being the layer normalisation (@ba2016layer), ie subtracting $\mu^l$ and dividing by $\sigma^l$ defined as:

$$
\mu^l = \frac{1}{H} \sum_{i=1}^H n_i^l \quad \sigma^l = \sqrt{\frac{1}{H} \sum_{i=1}^H (n_i^l - \mu^l)^2},
$$ {#eq-layernorm}

with $H$ denoting the number of $n$ hidden units in a layer $l$.

* Adding a layer's inputs to its outputs is also called "skip connection"
* The layer is then normalised [@ba2016layer] to avoid having the numbers grow too big, which is detrimental for gradient transmission
  * Layer normalisation uses the same computation both during training and inference times, and is particularly suitable for time series

In [52]:
def add_and_norm(
    x_list # List of input tensors (of the same dimension) for skip connection
    ):
    "Adds tensors with same dimensions and then normalises layer"
    tmp = layers.Add()(x_list)
    return layers.LayerNormalization()(tmp)

In [54]:
#| output: asis
#| echo: false
show_doc(add_and_norm, title_level=4)

---

#### add_and_norm

>      add_and_norm (x_list)

Adds tensors with same dimensions and then normalises layer

|    | **Details** |
| -- | ----------- |
| x_list | List of input tensors (of the same dimension) for skip connection |

#### Example usage

In [60]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 100

# input dimensions: batches / timesteps / features
x1 = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 
x2 = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

# output dimensions: batches / timesteps / features
x1x2 = add_and_norm(x_list=[x1, x2])
assert x1.shape == x1x2.shape

Mean values of the normalised layer should be around 0:

In [65]:
print("mean of x1 units at each batch X time step: ", x1.mean(axis=-1), "\n")
print("mean of x2 units at each batch X time step: ", x2.mean(axis=-1), "\n")
print("mean of sum of x1 and x2:"), x1x2.numpy().mean(axis=-1)

mean of x1 units at each batch X time step:  [[ 0.07455922  0.04061129  0.05181068 -0.09088949 -0.07739758]
 [ 0.01492953 -0.16913738 -0.17806861  0.02542704 -0.02922696]
 [-0.04603776  0.09231065 -0.07678928  0.01013863  0.01639121]] 

mean of x2 units at each batch X time step:  [[ 0.00980258 -0.00192174  0.06792247 -0.05294029 -0.06777942]
 [ 0.00694195 -0.1447223  -0.0799674   0.11319608  0.07691094]
 [-0.04790915 -0.06469555 -0.03416057 -0.04981646  0.18596057]] 

mean of sum of x1 and x2:


(None,
 array([[-2.3841858e-08, -1.1473894e-08,  9.5367430e-09,  2.3841857e-09,
         -7.1525572e-09],
        [-1.3113022e-08,  2.0861625e-08, -5.9604646e-09,  1.9073486e-08,
          5.9604646e-09],
        [-1.1920929e-08, -1.9073486e-08, -1.9073486e-08,  1.9073486e-08,
          5.8114527e-09]], dtype=float32))

Standard deviation (for the normalised output it should be around 1):

In [66]:
print("std of x1 units at each batch X time step: ", x1.std(axis=-1), "\n")
print("std of x2 units at each batch X time step: ", x2.std(axis=-1), "\n")
print("std of normalised sum of x1 and x2:"), x1x2.numpy().std(axis=-1)

std of x1 units at each batch X time step:  [[1.02569605 0.98900181 0.94005451 0.89980367 0.96177263]
 [1.17489033 1.05453288 0.90757409 0.92312574 0.94491605]
 [0.99893797 0.96701721 0.97790858 0.9491741  1.09512876]] 

std of x2 units at each batch X time step:  [[0.8787866  0.95885331 0.9797253  0.97875105 1.04979644]
 [0.96219559 0.92205835 1.03307451 0.88680021 0.93463915]
 [1.05767775 1.06415902 0.98560554 1.0963727  0.92135964]] 

std of normalised sum of x1 and x2:


(None,
 array([[0.9997369 , 0.99976975, 0.9997241 , 0.9997288 , 0.9997938 ],
        [0.999778  , 0.9997712 , 0.99977314, 0.9996005 , 0.99971753],
        [0.99974185, 0.99978566, 0.99975353, 0.99979323, 0.99974686]],
       dtype=float32))

### Gated linear unit (GLU)

> Linear layer that learns how much to gate vs let pass through

Using input $\gamma \in \mathbb{R}^{d_{\text{model}}}$ and the subscript $\omega$ to index weights, 

$$
\text{GLU}_{\omega}(\gamma) = \sigma(W_{4, \omega} \gamma + b_{4, \omega}) \odot (W_{5, \omega} \gamma + b_{5, \omega}).
$$ {#eq-GLU}

* Introduced by @dauphin2017language
* The intuition is to train two versions of @eq-dense in the same data, but one of them having a sigmoid activation (which outputs values between zero and one), then multiplying each hidden unit
* The result could be zero or very close to zero through the Hadamard multipliciation, which in practice means that the network would not be affected by that data (ie, it would be gated out)
  * The first term (with the sigmoid) is the gate that determines what percentage of the linear layer passes through
* *"GLUs reduce the vanishing gradient problem for deep architectures by providing a linear path for gradients while retaining non-linear capabilities"*
* *"provide flexibility to suppress any parts of the architecture that are not required for a given dataset"*
* The GLU is part of @sec-GRN

In [67]:
#| output: false

def apply_gating_layer(
    x, # Input tensors (batch first)
    hidden_layer_size:int, # Dimension of the GLU
    dropout_rate:float|None=None, # Dropout rate
    use_time_distributed:bool=True, # Apply the GLU across all timesteps?
    activation:str|callable=None # Activation function
): # Tuple of (GLU output tensors, gated_layer)
    "Gated Linear Unit (GLU) layer"
    
    if dropout_rate is not None:
        x = layers.Dropout(dropout_rate)(x)

    activation_layer = linear_layer(
        size=hidden_layer_size,
        activation=activation,
        use_time_distributed=use_time_distributed
    )(x)
    
    gate_layer = linear_layer(
        size=hidden_layer_size,
        activation='sigmoid',
        use_time_distributed=use_time_distributed
    )(x)

    return layers.Multiply()([activation_layer, gate_layer]), gate_layer

In [68]:
#| output: asis
#| echo: false
show_doc(apply_gating_layer, title_level=4)

---

#### apply_gating_layer

>      apply_gating_layer (x, hidden_layer_size:int,
>                          dropout_rate:Optional[float]=None,
>                          use_time_distributed:bool=True,
>                          activation:Union[str,<built-
>                          infunctioncallable>]=None)

Gated Linear Unit (GLU) layer

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| x |  |  | Input tensors (batch first) |
| hidden_layer_size | int |  | Dimension of the GLU |
| dropout_rate | float \| None | None | Dropout rate |
| use_time_distributed | bool | True | Apply the GLU across all timesteps? |
| activation | str \| callable | None | Activation function |

#### Example usage

In [69]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 100
hidden_layer_size = 16

# input dimensions: batches / timesteps / features
x = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

# output dimensions: batches / timesteps / hidden_layer_size
assert apply_gating_layer(x=x, hidden_layer_size=hidden_layer_size)[0].shape == [batch_size, n_timesteps, hidden_layer_size]

### Gated residual network (GRN) {#sec-GRN}

$$
\text{GRN}_{\omega}(a, c) = \text{LayerNorm}(a + \text{GLU}_{\omega}(W_{1, \omega} \text{ELU}(W_{2, \omega} a + b_{2, \omega} + W_{3, \omega} c) + b_{1, w}))
$$ {#eq-GRN}

* Breaking down $\text{GRN}_{\omega}(a, c)$:
    * *1st step*: $\eta_{2} = \text{ELU}(W_{2, \omega} a + b_{2, \omega} + W_{3, \omega} c)$ (where the additional context $c$ might be zero) as in @eq-dense but adapted for the added context if any and with $\text{ELU}(\cdot)$ as the activation function,
    * *2nd step*: $\eta_{1} = W_{1, \omega} \eta_{2} + b_{1, w}$ as in @eq-dense,
    * *3rd step*: $\text{LayerNorm}(a + \text{GLU}_{\omega}(\eta_{1}))$ as in @eq-skip and @eq-GLU
* $\text{ELU}(\cdot)$ is the Exponential Linear Unit activation function (@clevert2015fast)
    * Unlike ReLUs, ELUs allow for negative values, which pushes unit activations closer to zero at a lower computation complexity, and producing more accurate results
* The GRN is a key building block of the TFT
    * Helps keep information only from relevant input variables
    * Also keeps the model as simple as possible by only applying non-linearities when relevant

In [56]:
def gated_residual_network(
    x, # Network inputs
    hidden_layer_size:int, # Dimension of the GRN
    output_size:int|None=None, # Size of output layer (if None, same as `hidden_layer_size`)
    dropout_rate:float|None=None, # Dropout rate
    use_time_distributed:bool=True, # Apply the GRN across all timesteps?
    additional_context=None, # Additional context vector to use if relevant
    return_gate:bool=False #Whether to return GRN gate for diagnostic purposes
):
    "Applies the gated residual network (GRN) as defined in the paper"
    
    # Setup skip connection
    if output_size is None:
        output_size = hidden_layer_size
        skip = x
    else:
        linear = keras.layers.Dense(output_size)
        if use_time_distributed:
            linear = keras.layers.TimeDistributed(linear)
        skip = linear(x)

    # 1st step: eta2
    hidden = linear_layer(
        size=hidden_layer_size, # W2
        activation=None,
        use_time_distributed=use_time_distributed,
        use_bias=True # b2
    )(x)

    # "For instances without a context vector, the GRN simply treates the context input as zero - ie, $c = 0$ in Eq. 4"
    if additional_context is not None: # if c is != 0...
        hidden += linear_layer(
            size=hidden_layer_size, # W3
            activation=None,
            use_time_distributed=use_time_distributed,
            use_bias=False # no bias for additional context, since there already is bias from the "main" calculation of eta2
        )(additional_context)

    hidden = keras.layers.Activation('elu')(hidden)

    # 2nd step: eta1
    hidden = linear_layer(
        size=hidden_layer_size, # W1
        activation=None,
        use_time_distributed=use_time_distributed,
        use_bias=True # b1
    )(hidden)

    # 3rd step: concluding the GRN calculation
    gating_layer, gate = apply_gating_layer(
        x=hidden,
        hidden_layer_size=output_size,
        dropout_rate=dropout_rate,
        use_time_distributed=use_time_distributed,
        activation=None
    )

    GRN = add_and_norm([skip, gating_layer])

    if return_gate:
        return GRN, gate
    else:
        return GRN

In [57]:
#| output: asis
#| echo: false
show_doc(gated_residual_network, title_level=4)

---

#### gated_residual_network

>      gated_residual_network (x, hidden_layer_size:int,
>                              output_size:Optional[int]=None,
>                              dropout_rate:Optional[float]=None,
>                              use_time_distributed:bool=True,
>                              additional_context=None, return_gate:bool=False)

Applies the gated residual network (GRN) as defined in the paper

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| x |  |  | Network inputs |
| hidden_layer_size | int |  | Dimension of the GRN |
| output_size | int \| None | None | Size of output layer (if None, same as `hidden_layer_size`) |
| dropout_rate | float \| None | None | Dropout rate |
| use_time_distributed | bool | True | Apply the GRN across all timesteps? |
| additional_context | NoneType | None | Additional context vector to use if relevant |
| return_gate | bool | False | Whether to return GRN gate for diagnostic purposes |

#### Example usage

In [58]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 100
hidden_layer_size = 16
output_size = 17

# input dimensions: batches / timesteps / features
x = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

grn = gated_residual_network(
    x=x,
    hidden_layer_size=hidden_layer_size,
    output_size=output_size,
    dropout_rate=0,
    use_time_distributed=True,
    additional_context=None,
    return_gate=False
)

# output dimensions: batches / timesteps / hidden_layer_size
assert grn.shape == [batch_size, n_timesteps, output_size]

# Attention components

* Attention mechanisms use relationships between keys $K \in \mathbf{R}^{N \times d_{attention}}$ and queries $Q \in \mathbf{R}^{N \times d_{attention}}$ to scale a vector of values $V \in \mathbf{R}^{N \times d_V}$: $\text{Attention}(Q, K, V) = A(Q, K) V$
    * $N$ is the number of timesteps going into the attention layer (the number of lags plus the number of periods to be forecasted)
    * $A(\cdot)$ is a normalisation function
        * After @vaswani2017attention, the canonical choice for $A(\cdot)$ is the scaled dot-product: $A(Q, K) = \text{Softmax}(\frac{Q K^{T}}{\sqrt{d_{attention}}} )$
    
* The TFT uses a modified attention head to enhance the explainability of the model
* Specifically, the transformer block (multi-head attention) is modified to:
    * share values in each head, and
    * employ additive aggregation of all heads
* More formally, compare the interpretable multi-head attention (used in this paper) with the canonical multi-head attention:
    * $\text{InterpretableMultiHead}(Q, K, V) = \tilde{H} W_{H}$, with:
        * $\begin{aligned}\tilde{H} &= \tilde{A}(Q, K) V W_V \\
        &= \{\frac{1}{m_H} \sum^{m_{H}}_{h=1} A(Q W^{(h)}_Q, K W^{(h)}_K) \} V W_V \\
        &= \frac{1}{m_H} \sum^{m_{H}}_{h=1} \text{Attention}(Q W^{(h)}_Q, K W^{(h)}_K, V W_V)
        \end{aligned}$
    * $\text{MultiHead}(Q, K, V) = [H_1, \dots, H_{m_H}] W_H$, with:
        * $H_h = \text{Attention}(Q W^{(h)}_Q, K W^{(h)}_K, V W_V^{(h)}) $

## Decoder mask for self-attention layer

In [18]:
def get_decoder_mask(
    self_attention_inputs # Inputs to the self-attention layer
):
    "Determines shape of decoder mask"
    len_s = keras.ops.shape(self_attention_inputs)[1] # length of inputs
    bs = keras.ops.shape(self_attention_inputs)[0] # batch shape
    mask = keras.ops.cumsum(keras.ops.eye(len_s), 1) #keras.backend.cumsum(np.eye(len_s, bs))

    ### warning: I had to manually implement some batch-wise shape here 
    ### because the new keras `eye` function does not have a batch_size arg.
    ### inspired by: https://github.com/tensorflow/tensorflow/blob/v2.14.0/tensorflow/python/ops/linalg_ops_impl.py#L30
    ### <hack>
    mask = keras.ops.expand_dims(mask, axis=0)    
    mask = keras.ops.tile(mask, (bs, 1, 1))
    ### </hack>

    return mask


Example usage of the decoder mask:

In [19]:
dec = get_decoder_mask(grn)

assert dec.shape == (batch_size, n_timesteps, n_timesteps)

Note that it produces an upper-triangular matrix of ones:

In [20]:
dec[0]

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)>

In [21]:
show_doc(get_decoder_mask)

---

### get_decoder_mask

>      get_decoder_mask (self_attention_inputs)

Determines shape of decoder mask

|    | **Details** |
| -- | ----------- |
| self_attention_inputs | Inputs to the self-attention layer |

## Scaled dot product attention layer

* This is the same as Eq. (1) of @vaswani2017attention 
    * except that in this case the dimension of the value vector is the same $d_{\text{model}}$ as for the query and key vectors
* As discussed in the paper, additive attention outperforms dot product attention for larger $d_{\text{model}}$ values, so the attention is scaled back to smaller values

In [22]:
class ScaledDotProductAttention():
    def __init__(
        self,
        training:bool=True, # Whether the layer is being trained or used in inference
        attention_dropout:float=0.0 # Will be ignored if `training=False`
    ):
        self.training = training
        self.dropout = keras.layers.Dropout(rate=attention_dropout)
        self.activation = keras.layers.Activation('softmax')

    def __call__(
        self,
        q, # Queries, tensor of shape (?, time, D_model)
        k, # Keys, tensor of shape (?, time, D_model)
        v, # Values, tensor of shape (?, time, D_model)
        mask # Masking if required (sets Softmax to very large value), tensor of shape (?, time, time)
    ):
        # returns Tuple (layer outputs, attention weights)
        scale = keras.ops.sqrt(keras.ops.cast(keras.ops.shape(k)[-1], dtype='float32'))
        attention = keras.ops.einsum("bij,bjk->bik", q, keras.ops.transpose(k, axes=(0, 2, 1))) / scale
        if mask is not None:
            mmask = keras.layers.Lambda(lambda x: (-1e9) * (1. - keras.ops.cast(x, 'float32')))(mask)
            attention = keras.layers.Add()([attention, mmask])
        attention = self.activation(attention)
        if self.training:
            attention = self.dropout(attention)
        output = keras.ops.einsum("btt,btd->bt", attention, v)
        return output, attention

Below is an example of how the `ScaledDotProductAttention` layer works:

In [23]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 13

# input dimensions: batches / timesteps / features
x_btf = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

# using the same vector for q, k and v just to simplify
q=keras.ops.cast(x_btf, 'float32')
k=keras.ops.cast(x_btf, 'float32')
v=keras.ops.cast(x_btf, 'float32')

Testing without masking:

In [24]:
#| code-fold: show

output, attention = ScaledDotProductAttention()(q=q, k=k, v=v, mask=None)
output, attention # both have shape (batch_size, n_timesteps)

(<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
 array([[-1.3595526 ,  2.3038948 , -0.9351378 , -2.1107376 , -0.10875011],
        [ 0.74828297,  1.2987878 ,  2.7947266 ,  3.3696222 ,  0.02006484],
        [-4.0625644 ,  7.9136066 ,  0.15763263,  0.387267  ,  0.5300054 ]],
       dtype=float32)>,
 <tf.Tensor: shape=(3, 5, 5), dtype=float32, numpy=
 array([[[8.67120624e-01, 7.71935284e-02, 1.09112160e-02, 3.08754463e-02,
          1.38991298e-02],
         [1.48252323e-01, 7.48296022e-01, 6.03066683e-02, 2.28216760e-02,
          2.03233790e-02],
         [4.45502140e-02, 1.28209814e-01, 7.00278938e-01, 7.39124417e-02,
          5.30485511e-02],
         [2.03909084e-01, 7.84784630e-02, 1.19554065e-01, 5.13916612e-01,
          8.41417834e-02],
         [5.25127314e-02, 3.99808772e-02, 4.90878299e-02, 4.81354706e-02,
          8.10283065e-01]],
 
        [[9.24033284e-01, 1.75150018e-02, 4.11768891e-02, 4.70066769e-03,
          1.25742713e-02],
         [9.32998657e-02, 6.94437802e-01

... and with masking:

In [25]:
output, attention = ScaledDotProductAttention()(q=q, k=k, v=v, mask=get_decoder_mask(q))
output, attention # both have shape (batch_size, n_timesteps)

(<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
 array([[-1.3595526 ,  2.704903  , -1.130431  , -3.5293167 , -0.1342125 ],
        [ 0.74828297,  1.4324336 ,  3.4899209 ,  3.8354926 ,  0.03641403],
        [-4.0625644 ,  7.9578185 ,  0.22934216,  0.414706  ,  0.56441283]],
       dtype=float32)>,
 <tf.Tensor: shape=(3, 5, 5), dtype=float32, numpy=
 array([[[8.67120624e-01, 7.71935284e-02, 1.09112160e-02, 3.08754463e-02,
          1.38991298e-02],
         [0.00000000e+00, 8.78541887e-01, 7.08034411e-02, 2.67939400e-02,
          2.38607973e-02],
         [0.00000000e+00, 0.00000000e+00, 8.46524537e-01, 8.93482491e-02,
          6.41271621e-02],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.59308362e-01,
          1.40691578e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          1.00000000e+00]],
 
        [[9.24033284e-01, 1.75150018e-02, 4.11768891e-02, 4.70066769e-03,
          1.25742713e-02],
         [0.00000000e+00, 7.65895724e-01

In [26]:
show_doc(ScaledDotProductAttention)

---

### ScaledDotProductAttention

>      ScaledDotProductAttention (training:bool=True,
>                                 attention_dropout:float=0.0)

Initialize self.  See help(type(self)) for accurate signature.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| training | bool | True | Whether the layer is being trained or used in inference |
| attention_dropout | float | 0.0 | Will be ignored if `training=False` |

### Softmax

A small detour to illustrate the softmax function. 

The $i^{\text{th}}$ element of $\text{Softmax}(x)$, with $x \in \mathbf{R}^K$ is:

$$
\text{Softmax}(x)_i = \frac{e^{x_i}}{\sum_{j=1}^K e^{x_j}}
$$

For example, see the values below for an input vector $x$ ($K=5$ in this example):

In [27]:
#| code-fold: show

x = np.array([-np.Inf, -1., 0., 1., 3.])
keras.layers.Activation('softmax')(x)
print("x = ", x)
print("exp(x) = ", np.exp(x))
print("denominator (sum of exp(x_j), j=1,...,K) = ", sum(np.exp(x)))
print("softmax(x) = ", np.exp(x) / sum(np.exp(x)))
print("sum of softmax(x)_j, j=1,...,K = ", sum(np.exp(x) / sum(np.exp(x))))

x =  [-inf  -1.   0.   1.   3.]
exp(x) =  [ 0.          0.36787944  1.          2.71828183 20.08553692]
denominator (sum of exp(x_j), j=1,...,K) =  24.171698192818155
softmax(x) =  [0.         0.01521943 0.0413707  0.11245721 0.83095266]
sum of softmax(x)_j, j=1,...,K =  1.0


As can be seen above, the softmax function really makes the largest numbers stand out from the rest.

Note also that $-\infty$ results in 0.

## Interpretable Multi-head attention

* When values are shared in each head and then are aggregated additively, each head still lcan learn different temporal patterns (from their own unique queries and keys), but with the same input values.
    * In other words, they can be interpreted as an ensemble over the attention weights
    * the paper doesn't mention this explicitly, but the ensemble is equally-weighted - maybe there is some performance to be gained by having some way to weight the different attention heads 🤔, such as having a linear layer combining them... will explore in the future

In [71]:
class InterpretableMultiHeadAttention():
    def __init__(
        self,
        n_head:int,
        d_model:int,
        training:bool=True, # Whether the layer is being trained or used in inference
        dropout:float=0.0 # Will be ignored if `training=False`
    ):
        self.n_head = n_head
        self.d_k = self.d_v = d_k = d_v = d_model // n_head # the original model divides by number of heads
        self.training = training
        self.dropout = dropout

        # using the same value layer facilitates interpretability
        vs_layer = keras.layers.Dense(d_v, use_bias=False, name="Shared value")

        # creates list of queries, keys and values across heads
        self.qs_layers = self._build_layers(d_k, n_head)
        self.ks_layers = self._build_layers(d_k, n_head)
        self.vs_layers = [vs_layer for _ in range(n_head)]

        self.attention = ScaledDotProductAttention()
        self.w_o = keras.layers.Dense(d_v, use_bias=False, name="W_v") # W_v in Eqs. (14)-(16), output weight matrix to project internal state to the original TFT

    def __call__(
        self,
        q, # Queries, tensor of shape (?, time, D_model)
        k, # Keys, tensor of shape (?, time, D_model)
        v, # Values, tensor of shape (?, time, D_model)
        mask=None # Masking if required (sets Softmax to very large value), tensor of shape (?, time, time)
    ):
        heads = []
        attns = []
        for i in range(self.n_head):
            qs = self.qs_layers[i](q)
            ks = self.ks_layers[i](q)
            vs = self.vs_layers[i](v)
           
            head, attn = self.attention(qs, ks, vs, mask)
            if self.training:
                head = keras.layers.Dropout(self.dropout)(head)
            heads.append(head)
            attns.append(attn)

        outputs = keras.ops.mean(heads, axis=0) if self.n_head > 1 else head # H_tilde
        outputs = self.w_o(outputs)
        if self.training:
            outputs = keras.layers.Dropout(self.dropout)(outputs)

        return outputs, attn

    def _build_layers(self, d:int, n_head:int):
            return [keras.layers.Dense(d) for _ in range(n_head)]

In [72]:
imha = InterpretableMultiHeadAttention(n_head=8, d_model=16)

In [73]:
grn.shape # B, T, F

TensorShape([3, 5, 17])

In [74]:
mask = get_decoder_mask(grn)
mask.shape # B, T, T

TensorShape([3, 5, 5])

In [75]:
imha(grn, grn, grn, mask)

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 0.17549314, -0.6499947 ],
        [ 1.8635427 , -1.4109019 ],
        [ 0.5420394 , -1.9807158 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 5, 5), dtype=float32, numpy=
 array([[[0.00578043, 0.15875775, 0.20819457, 0.622656  , 0.00461125],
         [0.        , 0.16917567, 0.1295054 , 0.6177507 , 0.08356823],
         [0.        , 0.        , 0.03082285, 0.9643422 , 0.00483495],
         [0.        , 0.        , 0.        , 0.43220872, 0.56779134],
         [0.        , 0.        , 0.        , 0.        , 1.        ]],
 
        [[0.0467045 , 0.85227525, 0.00620111, 0.0769116 , 0.01790758],
         [0.        , 0.33351904, 0.46886626, 0.10064775, 0.09696695],
         [0.        , 0.        , 0.9063379 , 0.01591888, 0.07774319],
         [0.        , 0.        , 0.        , 0.74415594, 0.25584403],
         [0.        , 0.        , 0.        , 0.        , 1.        ]],
 
        [[0.10995895, 0.51384526, 0.00828304, 0.3354729 ,

# Time to build the TFT

In [107]:
class TemporalFusionTransformer():
    def __init__(
        self,
        # Data params
        time_steps:int,
        input_size:int,
        output_size:int,
        category_counts:int,
        n_workers:int, # Number of multiprocessing workers

        # TFT params
        input_obs_loc,
        static_input_loc,
        known_regular_input_idx,
        known_categorical_input_idx,
        column_definition,

        # Network params
        quantile:list=[0.1, 0.5, 0.9], # List of quantiles the model should forecast
        hidden_layer_size:int=30, # Size of hidden layer
        dropout_ratio:float=0.0, # Dropout ratio (between 0.0, inclusive, and less than 1.0)
        num_encoder_steps:int=4,
        num_stacks:int=4,
        num_heads:int=4,
        
        # Training params
        max_gradient_norm:float=1.0, # 
        learning_rate:float=0.001,
        minibatch_size:int=64,
        num_epochs:int=100,
        early_stopping_patience:int=5,
        use_gpu:bool=True
    ):
        self.time_steps = time_steps
        self.input_size = input_size
        self.output_size = output_size # Number of periods to be forecasted
        self.category_counts = category_counts
        self.n_workers = n_workers # Number of multiprocessing workers
        
        self.input_obs_loc = input_obs_loc
        self.static_input_loc = static_input_loc
        self.known_regular_input_idx = known_regular_input_idx
        self.known_categorical_input_idx = known_categorical_input_idx
        self.column_definition = column_definition

        self.quantile = quantile # List of quantiles the model should forecast
        self.hidden_layer_size = hidden_layer_size # Size of hidden layer
        self.dropout_ratio = dropout_ratio # Dropout ratio (between 0.0, inclusive, and less than 1.0)
        self.num_encoder_steps = num_encoder_steps
        self.num_stacks = num_stacks
        self.num_heads = num_heads
        
        self.max_gradient_norm = max_gradient_norm
        self.learning_rate = learning_rate
        self.minibatch_size = minibatch_size
        self.num_epochs = num_epochs
        self.early_stopping_patience = early_stopping_patience
        self.use_gpu = use_gpu

        self.model = self.build_model()

    def __static_combine_and_mask(
        self, 
        embedding # Transformed static inputs
    ):
        # Return the tensor output for the variable selection network
        # In the paper, that would be the bottom right square in Fig.2

        # Add temporal features
        _, num_static, _ = embedding.get_shape().as_list() # (embeddings are $\xi_t^(1, \dots, \m_{\chi})$)
        flattened = keras.layers.Flatten()(embedding) # $\Xi_t$

        # Nonlinear transformation with the GRN
        mlp_outputs = gated_residual_network(
            x=flattened, # Network inputs
            hidden_layer_size=self.hidden_layer_size, # Dimension of the GRN
            output_size=num_static, # Size of output layer (if None, same as `hidden_layer_size`)
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=False, # Apply the GRN across all timesteps?
            additional_context=None, # Additional context vector to use if relevant
        ) 
        sparse_weights = keras.layers.Activation('softmax')(mlp_outputs)
        sparse_weights = keras.ops.expand_dims(sparse_weights, axis=-1) # $\upsilon_{\chi t}$
        # it's the sparse weights above that determine by how much the variable will be influencing the model

        transformed_embeddings = []
        for i in range(num_static):
            transformed_embeddings.append(gated_residual_network(
                x=embedding[:, i:i+1, :], # Network inputs
                hidden_layer_size=self.hidden_layer_size, # Dimension of the GRN
                output_size=self.hidden_layer_size, # Size of output layer (if None, same as `hidden_layer_size`)
                dropout_rate=self.dropout_rate, # Dropout rate
                use_time_distributed=False, # Apply the GRN across all timesteps?
            ))
        transformed_embedding = keras.ops.concatenate(transformed_embeddings, axis=1) # $\tilde{\xi_t^(1, \dots, \m_{\chi})}$

        combined = keras.layers.Multiply()(
            [sparse_weights, transformed_embedding]
        )
        
        static_vec = keras.ops.sum(combined, axis=1)

        return static_vec, sparse_weights

    def __lstm_combine_and_mask(
        self,
        embedding # Transformed inputs
    ):
        # Temporal variable selection networks

        # Add temporal features
        _, time_steps, embedding_dim, num_inputs = embedding.get_shape().as_list()

        flattened = keras.ops.reshape(
            embedding,
            [1, time_steps, embedding_dim * num_inputs]
        )
        expanded_static_context_c_s = keras.ops.expand_dims(
            self.static_context_variable_selection,
            axis=1
        )

        # Variable selection weights
        mlp_outputs, static_gate = gated_residual_network(
            x=flatten,
            hidden_layer=self.hidden_layer_size,
            output_size=num_inputs,
            dropout_rate=self.dropout_rate,
            use_time_distributed=True,
            additional_context=expanded_static_context_c_s,
            return_gate=True
        )
        sparse_weights = keras.layers.Activation('softmax')(mlp_outputs)
        sparse_weights = keras.ops.expand_dims(sparse_weights, axis=2)

        # Nonlinear processing and application of weights
        transformed_embeddings = []
        for i in range(num_inputs):
            transformed.embeddings.append(
                gated_residual_network(
                    embedding[Ellipsis, i],
                    hidde_layer=self.hidden_layer_size,
                    dropout_rate=self.dropout_rate,
                    use_time_distributed=True
                )
            )
        transformed_embeddings = keras.ops.stack(transformed_embeddings, axis=-1)

        combined = keras.layers.Multiply()([
            sparse_weights, transformed_embedding
        ])
        temporal_ctx = keras.ops.sum(combined, axis=-1)

        return temporal_ctx, sparse_weights, static_gate

    def ___convert_real_to_embedding(
        self,
        x # Inputs
    ):
        return keras.layers.TimeDistributed(
            keras.layers.Dense(self.hidden_layer_size)(x)
        )

    def __get_tft_embeddings(
        self,
        all_inputs
    ):
        # Transform raw inputs to embeddings
        # For continuous variables: linear transformation
        # For categorical variables: embeddings
        
        num_categorical_variables = len(self.category_counts)
        num_regular_variables = self.input_size - num_categorical_variables

        embedding_sizes = [
            self.hidden_layer_size
            for i, size in enumerate(self.category_counts)
        ]

        embeddings = [
            keras.Sequential([
                keras.layers.InputLayer([time_steps]),
                keras.layers.Embedding(
                    self.category_counts[i],
                    embedding_sizes[i],
                    input_length=time_steps,
                    dtype='float32'
                )
            ])
            for i in range(num_categorical_variables)
        ]

        regular_inputs, categorical_inputs = \
            all_inputs[:, :, :num_regular_varaibles], \
            all_inputs[:, :, num_regular_variables:]

        embedded_inputs = [
            embeddings[i](categorical_inputs[Ellipsis, i])
            for i in range(num_categorical_variables)
        ]

        # static inputs
        if self._static_input_loc:
            st_inp_dense = [
                keras.layers.Dense(self.hidden_layer_size)(
                    regular_inputs[:, 0, i:i + 1]
                )
                for i in range(num_regular_variables)
                if i in self._static_input_loc
            ]
            st_inp_embed = [
                embedded_inputs[i][:, 0, :]
                for i in range(num_categorical_variables)
                if  i + num_regular_variables in self._static_input_loc
            ]
            static_inputs = st_inp_dense + st_inp_embed
        else:
            static_inputs = None

        # Targets
        obs_inputs = keras.ops.stack([
            self.___convert_real_to_embedding(regular_inputs[Ellipsis, i:i + 1])
        ], axis=-1)

        # past inputs: observed but not known a priori
        wired_embeddings = [
            embeddings[i](categorical_inputs[:,:,i])
            for i in range(num_categorical_variables)
            if i not in self._known_categorical_input_idx \
                and i + num_regular_variables not in self._input_obs_loc    
        ]
        unknown_inputs = [
            self.___convert_real_to_embedding(regular_inputs[Ellipsis, i:i + 1])
            for i in range(regular_inputs.shape[-1])
            if i not in self._known_categorical_input_idx \
                and i + num_regular_variables not in self._input_obs_loc    
        ]
        if wired_embeddings + unknown_inputs:
            unknown_inputs = keras.ops.stack(wired_embeddings + unknown_inputs, axis=-1)
        else:
            unkown_inputs = None

        # a priori known inputs
        known_regular_inputs = [
            self.___convert_real_to_embedding(regular_inputs[Ellipsis, i:i + 1])
            for i in self._known_regular_input_idx
            if i not in self._static_input_loc
        ]
        known_categorical_inputs = [
            embedded_inputs[i]
            for i in self._known_categorical_input_idx
            if i + num_regular_variables not in self._static_input_loc
        ]
        known_combined_layer = keras.ops.stack(
            known_regular_inputs + known_categorical_inputs,
            axis=-1
        )

        return unknown_inputs, known_combined_layer, past_inputs, static_inputs

    def _build_base_graph(self):
        # Build the graph, defining the layers of the TFT
        
        all_inputs = keras.layers.Input(
            shape=(self.time_steps, self.input_size)
        )
        unknown_inputs, known_combined_layer, past_inputs, static_inputs \
            = self.__get_tft_embeddings(all_inputs)

        # first we isolate the known future inputs and observed past inputs
        if unknown_inputs is not None:
            historical_inputs = keras.ops.concatenate([
                unknown_inputs[:, :self.num_encoder_steps, :],
                known_combined_layer[:, :self.num_encoder_steps, :],
                past_inputs[:, :self.num_encoder_steps, :]
            ], axis=1)
        else:
            historical_inputs = keras.ops.concatenate([
                known_combined_layer[:, :self.num_encoder_steps, :],
                past_inputs[:, :self.num_encoder_steps, :]
            ])
        
        # and then we isolate the known future inputs
        future_inputs = known_combined_layer[:, :self.num_encoder_steps, :]

        # static vars
        static_encoder, static_weights = self.__static_combine_and_mask(static_inputs)

        # Static covariate encoders
        # These integrate static features into the network through encoding of context vectors
        # that condition the time-varying dynamics
        self.static_context_variable_selection = gated_residual_network( # c_s
            x=static_encoder, # Network inputs
            hidden_layer_size=self.hidden_layer_size, # Dimension of the GRN
            output_size=self.hidden_layer_size, # Size of output layer (if None, same as `hidden_layer_size`)
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=False, # Apply the GRN across all timesteps?
        )
        self.static_context_enrichment = gated_residual_network( # c_3
            x=static_encoder, # Network inputs
            hidden_layer_size=self.hidden_layer_size, # Dimension of the GRN
            output_size=self.hidden_layer_size, # Size of output layer (if None, same as `hidden_layer_size`)
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=False, # Apply the GRN across all timesteps?
        )
        self.static_context_state_h = gated_residual_network( # c_h
            x=static_encoder, # Network inputs
            hidden_layer_size=self.hidden_layer_size, # Dimension of the GRN
            output_size=self.hidden_layer_size, # Size of output layer (if None, same as `hidden_layer_size`)
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=False, # Apply the GRN across all timesteps?
        )
        self.static_context_state_c = gated_residual_network( # c_c
            x=static_encoder, # Network inputs
            hidden_layer_size=self.hidden_layer_size, # Dimension of the GRN
            output_size=self.hidden_layer_size, # Size of output layer (if None, same as `hidden_layer_size`)
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=False, # Apply the GRN across all timesteps?
        )

        historical_features, historical_flags, _ = self.__lstm_combine_and_mask(historical_inputs)
        future_features, future_flags, _ = self.__lstm_combine_and_mask(future_inputs)

        # Locality enhancement (Section 4.5.1 in paper) with seq-to-seq layer

        # LSTM layers: LSTM Encoder for encoding past inputs
        history_lstm, state_h, state_c = keras.layers.LSTM(
            units=self.hidden_layer_size,
            return_sequences=True,
            return_state=True,
            stateful=False,
            activation='tanh',
            recurrent_activation='sigmoid',
            recurrent_dropout=0,
            unroll=False,
            use_bias=True)(
                inputs=historical_features,
                initial_state=[
                    self.static_context_state_h, # short-term state
                    self.static_context_state_c  # long-term state
                ]
            )

        # LSTM layers: LST Decoder for decoding future inputs
        future_lstm = keras.layers.LSTM(
            units=self.hidden_layer_size,
            return_sequences=True,
            return_state=False,
            stateful=False,
            activation='tanh',
            recurrent_activation='sigmoid',
            recurrent_dropout=0,
            unroll=False,
            use_bias=True)(
                inputs=future_features,
                initial_stage=[
                    state_h, # short-term state
                    state_c  # long-term state
                ]
            )

        lstm_layer = keras.ops.concatenate([history_lstm, future_lstm], axis=1)

        # Apply gated skip connection (Gate followed by Add & Norm)

        input_embeddings = keras.ops.concatenate(
            [historical_features, future_features],
            axis=1
        )
        lstm_layer, _ = apply_gating_layer(
            x=lstm_layer, # Input tensors (batch first)
            hidden_layer_size=self.hidden_layer_size, # Dimension of the GLU
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=True, # Apply the GLU across all timesteps?
            activation=None # Activation function
        )
        temporal_feature_layer = add_and_norm([lstm_layer, input_embeddings])

        # Temporal Fusion Decoder (TFT, Purple box in Fig. 2)
        # contains three steps
        # TFT 1st step: Static enrichment
        #   - enhances the temporal features with static metadata (Eq. 18)
        
        expanded_static_context_c_e = keras.ops.expand_dims(
            static_context_enrichment,
            axis=1
        )
        enriched, _ = gated_residual_network( # $\theta(t, n) = \text{GRN}_{\theta}(\tilde{\theta}(t, n), c_e)
            x=temporal_feature_layer,
            hidden_layer_size=self.hidden_layer_size,
            output_size=self.hidden_layer_size,
            dropout_rate=self.dropout_rate,
            use_time_distributed=True,
            additional_context=expanded_static_context_c_e,
            return_gate=True
        )

        # TFT 2nd step: Temporal self-attention

        self_attention_layer = InterpretableMultiHeadAttention(
            n_head=self.num_heads,
            d_model=self.hidden_layer_size,
            dropout=self.dropout_rate # Will be ignored if `training=False`
        )
        mask = get_decoder_mask(enriched)
        post_attn, self_attention = self_attention_layer( # $B(t) = \text{IMHA}(\Theta(t), \Theta(t), \Theta(t))$
            q=enriched,
            k=enriched,
            v=enriched,
            mask=mask
        )
        post_attn, _ = apply_gating_layer( # $\text{GLU}_{\delta}(\beta(t, n))$
            x=post_attn, # Input tensors (batch first)
            hidden_layer_size=self.hidden_layer_size, # Dimension of the GLU
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=True, # Apply the GLU across all timesteps?
            activation=None # Activation function
        )
        post_attn = add_and_norm([post_attn, enriched]) # \delta(t, n) = \text{LayerNorm}(\theta(t, n) + $\text{GLU}_{\delta}(\beta(t, n)))$

        # TFT 3rd step: Position-wise feed-forward
        decoder = gated_residual_network(
            x=post_attn,
            hidden_layer_size=self.hidden_layer_size,
            output_size=self.hidden_layer_size,
            dropout_rate=self.dropout_rate,
            use_time_distributed=True,
            additional_context=None,
            return_gate=False
        )

        # final skip connection
        decoder, _ = apply_gating_layer(
            x=decoder, # Input tensors (batch first)
            hidden_layer_size=self.hidden_layer_size, # Dimension of the GLU
            dropout_rate=self.dropout_rate, # Dropout rate
            activation=None # Activation function
        )
        transformer_layer = add_and_norm([decoder, temporal_future_layer])

        # the function also returns the attention components
        # for explainability analyses
        attention_components = {
            "temporal_attention_weights": self_attention,
            "variable_selection_weights_static_inputs": static_weights[Ellipsis, 0],
            "variable_selection_weights_past_inputs": historical_flags[Ellipsis, 0, :],
            "variable_selection_weights_future_inputs": future_flags[Ellipsis, 0, :]
        }

        return transformer_layer, all_inputs, attention_components

    def build_model(self):
        # Build model and define training losses

        transformer_layer, all_inputs, self._attention_components = self._build_base_graph()
        outputs = keras.layers.TimeDistributed(
            keras.layers.Dense(self.output_size * len(self.quantiles))
        )(transformer_layer[Ellipsis, self.num_encoder_steps:, :])
        model = keras.Model(inputs=all_inputs, outputs=outputs)

In [108]:
tft = TemporalFusionTransformer(
    time_steps=12,
    input_size=20,
    output_size=4,
    category_counts=5,
    n_workers=2, # Number of multiprocessing workers

    # TFT params
    input_obs_loc=24,
    static_input_loc=24,
    known_regular_input_idx=24,
    known_categorical_input_idx=24,
    column_definition=None,
)

TypeError: object of type 'int' has no len()

In [97]:
tft.time_steps

12

# References {.unnumbered}